In [1]:
!CUDA_LAUNCH_BLOCKING=1
!nvidia-smi


Tue Oct  5 01:33:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -r /content/drive/MyDrive/textG/requirements.txt

In [3]:
import os
import sys
sys.path.append('/content/drive/MyDrive/textG')
print(os.getcwd())



/content


In [4]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from model.torch_gpt2 import GPT2Config, GPT2LMHeadModel # model폴더의 torch_gpt2.py의 
# from transformers.configuration_gpt2 import GPT2Config 를 from transformers import GPT2Config로 변경해 오류 잡음
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from util.data import FairyDataset
import gluonnlp
from tqdm import tqdm


In [5]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='/content/drive/MyDrive/textG/kogpt2/' # KoGPT-2 모델 다운로드 경로
save_path = '/content/drive/MyDrive/textG/checkpoint/'
use_cuda = True # Colab내 GPU 사용을 위한 값

kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 51200
}

In [6]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
kogpt2model.from_pretrained("skt/kogpt2-base-v2")

# 추가로 학습하기 위해 .train() 사용
kogpt2model.train()
vocab_b_obj = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='<s>', eos_token='</s>', unk_token='<unk>',  pad_token='<pad>', mask_token='<mask>') 
 


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
model, vocab = kogpt2model, vocab_b_obj.get_vocab()
sentencepieceTokenizer = vocab_b_obj.tokenize
#os.chdir("../")
# data_file_path = '/content/drive/MyDrive/Colab Notebooks/narrativeKoGPT2/data/backmyo_novel_1/dataset_0825.txt'
data_file_path = '/content/drive/MyDrive/Colab Notebooks/create_data/use_data/구텐베르크 동화/total_data.txt'
# /content/drive/MyDrive/Colab Notebooks/create_data/use_data/dataset_2.txt 이솝우화 약 312개
dataset = FairyDataset(data_file_path, vocab, sentencepieceTokenizer)

In [30]:
batch_size = 4 # batch_size 1이면 오류안남 2면 오류남/ 고침
fairy_data_loader = DataLoader(dataset, batch_size=batch_size)

In [31]:
# 1e-4 5e-5 2.5e-5 2e-5
learning_rate = 1e-4 # 학습률 잠시 수정 원래는 1e-5임
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
# 메모리 오류 나면 사용
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [33]:
model.cuda()
print('KoGPT-2 Transfer Learning Start')
epochs=200
# 학습률에 따른 에폭 조정해서 돌아가도록 코드 수정해야함
for epoch in range(epochs):
    count = 0
    print(epoch)
    for data in fairy_data_loader:
        optimizer.zero_grad()
        data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
        data= data.transpose(1,0)
        
        data= data.to(ctx)
   
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        if count %10 ==0:
            print('epoch no.{} train no.{}  loss = {}' . format(epoch, count+1, loss))
            # torch.save(model,save_path+'checkpoint_{}_{}.tar'.format(epoch,count))
            # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기

        count += 1

print("save!")
save_path = '/content/drive/MyDrive/Colab Notebooks/narrativeKoGPT2/checkpoint/'       
torch.save({
        'epoch': epoch,
        'train_no': count,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':loss
    }, save_path+'gt_checkpoint_2.tar')

KoGPT-2 Transfer Learning Start
0
epoch no.0 train no.1  loss = 7.76411247253418
epoch no.0 train no.11  loss = 8.240137100219727
epoch no.0 train no.21  loss = 8.099803924560547
epoch no.0 train no.31  loss = 8.464111328125
1
epoch no.1 train no.1  loss = 7.237978935241699
epoch no.1 train no.11  loss = 7.607227325439453
epoch no.1 train no.21  loss = 7.618948459625244
epoch no.1 train no.31  loss = 8.079451560974121
2
epoch no.2 train no.1  loss = 7.025771617889404
epoch no.2 train no.11  loss = 7.274022102355957
epoch no.2 train no.21  loss = 7.264321804046631
epoch no.2 train no.31  loss = 7.755885601043701
3
epoch no.3 train no.1  loss = 6.672304630279541
epoch no.3 train no.11  loss = 6.976747512817383
epoch no.3 train no.21  loss = 6.92443323135376
epoch no.3 train no.31  loss = 7.436278343200684
4
epoch no.4 train no.1  loss = 6.395531177520752
epoch no.4 train no.11  loss = 6.638428211212158
epoch no.4 train no.21  loss = 6.588153839111328
epoch no.4 train no.31  loss = 7.1452